In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [5]:
from trigger_utils.trigger_utils import get_kowalski_ztf_queue
import yaml

with open('../config/Credentials.yaml', 'r') as file:
    credentials = yaml.safe_load(file)
fritz_token = credentials['fritz_token']
ztf_allocation = credentials['allocation']

In [7]:
# print the current queue

current_ztf_queue = get_kowalski_ztf_queue(fritz_token, ztf_allocation)
current_ztf_queue

{'status': 'success',
 'data': {'queue_names': ['Caltech_Prince_2025-05-01',
   'Caltech_Prince_2025-05-03',
   'EP_2025-04-30_12',
   'EP_2025-04-30_14',
   'EP_2025-04-30_15',
   'EP_2025-04-30_16',
   'EP_2025-04-30_5',
   'EP_2025-04-30_6',
   'EP_2025-04-30_8',
   'EP_2025-04-30_9',
   'ToO_S250319bu_BBHBot_2025-05-01 01:25:32.352',
   'Twilight_2025-04-30_e',
   'Twilight_2025-04-30_m',
   'Twilight_2025-05-01_e',
   'Twilight_2025-05-01_m',
   'Twilight_2025-05-02_e',
   'Twilight_2025-05-02_m',
   'Twilight_2025-05-03_e',
   'Twilight_2025-05-03_m',
   'Twilight_2025-05-04_e',
   'Twilight_2025-05-04_m',
   'default',
   'fallback',
   'missed_obs']},
 'version': '1.4.0+fritz.86bd000'}

In [10]:
# get plan stats

from trigger_utils.trigger_utils import get_plan_stats

# get a specific plan
gcnevent_id=13342
queuename="S250319bu_BBHBot_2025-04-03 01:18:21.260"
stats = get_plan_stats(gcnevent_id, queuename, fritz_token, mode='')

Already submitted to queue
Total time: 2940, probability: 0.7988238394848364


In [ ]:
# backtracking NUZTF

# scanner.plot_coverage() prints 'In total, __% of the contour was observed at least once' etc
# scanner comes from scanner = SkymapScanner(event=skymap_file_url,prob_thresshold=0.9,n_days=3)
# SkymapScanner is in nuztf.skymap_scanner
# plot_coverage uses the function plot_overlap_with_observations(first_det_window_days=self.n_days,fields=fields(list=None))
# plot_overlap_with_observations is in base_scanner.py
# plot_overlap_with_observations uses the function calculate_overlap_with_observations

# functions to track down for calculate_overlap_with_observations:
# get_obs_summary
# get_nested_pix

In [1]:
import pandas as pd
import numpy as np
import healpy as hp
import astropy.units as u
from tqdm import tqdm
from astropy.coordinates import SkyCoord
from astropy.time import Time

In [ ]:
"""
MNS class
"""

import pandas as pd

class MNS:
    def __init__(self, df: pd.DataFrame):
        self.data = df

In [ ]:
def get_coverage(jds: [int], backend="best") -> pd.DataFrame | None:
    """
    Get a dataframe of the coverage for a list of JDs

    Will use the cache if available, otherwise will query the depot, and lastly TAP

    :param jds: JDs
    :param backend: "best" or "depot" or "tap" or "skyvision" or "masterlog"
    :return: Coverage dataframe
    """

    assert backend in [
        "best",
        "depot",
        "tap",
        "skyvision",
        "masterlog",
    ], f"Invalid backend '{backend}'"

    # Clear any logs flagged as partial/incomplete

    cache_files = coverage_dir.glob("*.json")
    partial_logs = [x for x in cache_files if partial_flag in str(x)]

    if len(partial_logs) > 0:
        print(f"Removing the following partial logs: {partial_logs}")
        for partial_log in partial_logs:
            partial_log.unlink()

    # Only write missing logs

    covered_jds = []

    if backend in ["best", "depot"]:
        for jd in jds:
            if jd not in covered_jds:
                depot_path = coverage_depot_path(jd)
                if depot_path.exists():
                    df = pd.read_json(depot_path)
                    if len(df) > 0:
                        covered_jds.append(jd)

        missing_logs = sorted(set(jds) - set(covered_jds))

        if len(missing_logs) > 0:
            print(
                f"Some logs were missing from the cache. "
                f"Querying for the following JDs in depot: {missing_logs}"
            )
            write_coverage_depot(missing_logs)

    if backend in ["best", "skyvision"]:
        
        for jd in jds:
            if jd not in covered_jds:
                skyvision_path = coverage_skyvision_path(jd)
                if skyvision_path.exists():
                    df = pd.read_json(coverage_skyvision_path(jd))
                    if len(df) > 0:
                        covered_jds.append(jd)

        missing_logs = sorted(set(jds) - set(covered_jds))

        if len(missing_logs) > 0:
            print(
                f"Some logs were still missing from the cache. "
                f"Querying for the following JDs from skyvision: {missing_logs}"
            )
            write_coverage_skyvision(missing_logs)

    # Try TAP for missing logs

    if backend in ["best", "tap"]:
        for jd in jds:
            if jd not in covered_jds:
                tap_path = coverage_tap_path(jd)
                if tap_path.exists():
                    df = pd.read_json(tap_path)
                    if len(df) > 0:
                        covered_jds.append(jd)

        missing_logs = sorted(set(jds) - set(covered_jds))

        if len(missing_logs) > 0:
            print(
                f"Some logs were still missing from the cache. "
                f"Querying for the following JDs from TAP: {missing_logs}"
            )
            write_coverage_tap(missing_logs)

    # Load logs from cache

    results = []

    for jd in tqdm(jds):

        res = None

        if backend in ["best", "depot"]:
            df = pd.read_json(coverage_depot_path(jd))
            if len(df) > 0:
                res = df

        if backend in ["best", "skyvision"]:
            if res is None:
                df = pd.read_json(coverage_skyvision_path(jd))
                if len(df) > 0:
                    res = df

        if backend in ["best", "tap"]:
            if res is None:
                df = pd.read_json(coverage_tap_path(jd))
                if len(df) > 0:
                    res = df

        if res is not None:
            results.append(res)

    if results:
        result_df = pd.concat(results, ignore_index=True)
        return result_df
    else:
        return None

In [ ]:
def get_obs_summary_depot(t_min: Time, t_max: Time, backend="best") -> MNS | None:
    """
    Get observation summary from depot

    :param t_min: Start time
    :param t_max: End time
    :param backend: "best" or "depot" or "tap" or "skyvision"
    :return: MNS object
    """

    jds = np.arange(int(t_min.jd) - 0.5, int(t_max.jd) + 1.5)

    res = get_coverage(jds, backend=backend)

    if len(res) == 0:
        return None

    res["date"] = Time(res["obsjd"].to_numpy(), format="jd").isot

    mns = MNS(df=res)

    mns.data.query(f"obsjd >= {t_min.jd} and obsjd <= {t_max.jd}", inplace=True)

    mns.data.reset_index(inplace=True)
    mns.data.drop(columns=["index"], inplace=True)

    return mns

In [ ]:
def get_obs_summary(
    t_min,
    t_max=None,
    max_days: float = None,
    backend="best",
) -> MNS | None:
    """
    Get observation summary from IPAC depot

    :param t_min: Start time
    :param t_max: End time
    :param max_days: Maximum number of days
    :param backend: "best" or "depot" or "tap" or "skyvision"
    :return: MNS object
    """
    now = Time.now()

    if t_max and max_days:
        raise ValueError("Choose either t_max or max_days, not both")

    if t_max is None:
        if max_days is None:
            t_max = now
        else:
            t_max = t_min + (max_days * u.day)

    if t_max > now:
        t_max = now

    print(f"Getting observation logs  using backend {backend}.")
    mns = get_obs_summary_depot(t_min=t_min, t_max=t_max, backend=backend)

    if mns is not None:
        print(f"Found {len(set(mns.data))} observations in total.")
    else:
        print(f"Found no observations using backend {backend}.")

    return mns

In [ ]:
# to define in class: t_min, nside, pixel_nos
# remove self.logger

def calculate_overlap_with_observations(
    self,
    first_det_window_days: float = 3.0,
    min_sep: float = 0.01,
    backend: str = "best",
):
    """
    Calculate the overlap of the skymap with observations

    :param first_det_window_days: First detection window in days
    :param min_sep: Minimum separation between detections in days
    :param backend: Backend to use for coverage calculation
    :return:
    """

    mns = get_obs_summary(
        t_min=self.t_min, max_days=first_det_window_days, backend=backend
    )

    if mns is None:
        return None, None, None

    data = mns.data.copy()

    mask = data["status"] == 0

    print(
        f"Found {mask.sum()} successful observations in the depot, "
        f"corresponding to {np.mean(mask)*100:.2f}% of the total."
    )

    pix_map = dict()
    pix_obs_times = dict()

    nested_pix = get_nested_pix(nside=self.nside, logger=self.logger)

    for i, obs_time in enumerate(tqdm(list(set(data["obsjd"])))):
        obs = data[data["obsjd"] == obs_time]

        field = obs["field_id"].iloc[0]

        try:
            flat_pix = nested_pix[int(field)]

            mask = obs["status"].astype(int) == 0
            indices = obs["qid"].values[mask]

            for qid in indices:
                pixels = flat_pix[int(qid)]

                for p in pixels:
                    if p not in pix_obs_times.keys():
                        pix_obs_times[p] = [obs_time]
                    else:
                        pix_obs_times[p] += [obs_time]

                    if p not in pix_map.keys():
                        pix_map[p] = [field]
                    else:
                        pix_map[p] += [field]

        except (KeyError, ValueError):
            print(
                f"Field {field} not found in nested pix dict. "
                f"This might be an engineering observation."
            )

    npix = hp.nside2npix(self.nside)

    ras, decs = hp.pixelfunc.pix2ang(
        self.nside, hp.nest2ring(self.nside, self.pixel_nos), lonlat=True
    )

    radecs = SkyCoord(ra=ras * u.deg, dec=decs * u.deg)

    coverage_data = []

    overlapping_fields = []
    times = []

    for i, p in enumerate(tqdm(hp.nest2ring(self.nside, self.pixel_nos))):
        entry = {
            "pixel_no": p,
            "prob": self.map_probs[i],
            "gal_b": radecs[i].galactic.b.deg,
            "in_plane": abs(radecs[i].galactic.b.deg) < 10.0,
            "ra_deg": ras[i],
            "dec_deg": decs[i],
        }

        if p in pix_obs_times.keys():
            obs = pix_obs_times[p]

            entry["n_det_class"] = 2 if max(obs) - min(obs) > min_sep else 1

            entry["latency_days"] = min(obs) - self.t_min.jd

            overlapping_fields += pix_map[p]

            times += list(obs)
        else:
            entry["n_det_class"] = 0
            entry["latency_days"] = np.nan

        coverage_data.append(entry)

    coverage_df = pd.DataFrame(coverage_data)

    overlapping_fields = sorted(list(set(overlapping_fields)))

    _observations = data.query("obsjd in @times").reset_index(drop=True)[
        ["obsjd", "exposure_time", "filter_id"]
    ]
    bands = [self.fid_to_band(fid) for fid in _observations["filter_id"].values]
    _observations["band"] = bands
    _observations.drop(columns=["filter_id"], inplace=True)
    self.observations = _observations

    self.logger.info("All observations:")
    self.logger.info(f"\n{self.observations}")

    try:
        self.first_obs = Time(min(times), format="jd")
        self.first_obs.utc.format = "isot"
        self.last_obs = Time(max(times), format="jd")
        self.last_obs.utc.format = "isot"

    except ValueError:
        err = (
            f"No observations of this event were found at any time between "
            f"{self.t_min} and {self.t_min + first_det_window_days * u.day}. "
            f"Coverage overlap is 0%!"
        )
        self.logger.error(err)
        raise ValueError(err)

    self.logger.info(f"Observations started at {self.first_obs.isot}")

    return (
        coverage_df,
        times,
        overlapping_fields,
    )

In [ ]:
def plot_overlap_with_observations(
        self, first_det_window_days=None, min_sep=0.01, fields=None, backend="best"
    ):
        """
        Function to plot the overlap of the field with observations.

        :param first_det_window_days: Window of time in days to consider for the first detection.
        :param min_sep: Minimum separation between observations to consider them as separate.
        :param fields: Fields to consider.
        :param backend: Backend to use for coverage calculation

        """

        (
            coverage_df,
            times,
            overlapping_fields,
        ) = self.calculate_overlap_with_observations(
            first_det_window_days=first_det_window_days,
            min_sep=min_sep,
            fields=fields,
            backend=backend,
        )

        if coverage_df is None:
            self.logger.warning("Not plotting overlap with observations.")
            return

        fig = plt.figure()
        plt.subplot(projection="aitoff")

        self.overlap_fields = list(set(overlapping_fields))

        overlap_mask = (coverage_df["n_det_class"] == 2) & ~coverage_df["in_plane"]
        self.overlap_prob = coverage_df[overlap_mask]["prob"].sum() * 100.0

        size = hp.max_pixrad(self.nside) ** 2 * 50.0

        veto_pixels = coverage_df.where(coverage_df["n_det_class"] == 0)

        if len(veto_pixels) > 0:
            plt.scatter(
                np.radians(veto_pixels["ra_deg"]),
                np.radians(veto_pixels["dec_deg"]),
                color="red",
                s=size,
            )

        plane_pixels = coverage_df.where(
            coverage_df["in_plane"] & (coverage_df["n_det_class"] > 0)
        )

        if len(plane_pixels) > 0:
            plt.scatter(
                np.radians(self.wrap_around_180(plane_pixels["ra_deg"])),
                np.radians(plane_pixels["dec_deg"]),
                color="green",
                s=size,
            )

        single_pixels = coverage_df.where(
            ~coverage_df["in_plane"] & (coverage_df["n_det_class"] == 1)
        )

        if len(single_pixels) > 0:
            plt.scatter(
                np.radians(self.wrap_around_180(single_pixels["ra_deg"])),
                np.radians(single_pixels["dec_deg"]),
                c=single_pixels["prob"],
                vmin=0.0,
                vmax=max(self.data[self.key]),
                s=size,
                cmap="gray",
            )

        double_pixels = coverage_df.where(
            ~coverage_df["in_plane"] & (coverage_df["n_det_class"] == 2)
        )

        if len(double_pixels) > 0:
            plt.scatter(
                np.radians(self.wrap_around_180(double_pixels["ra_deg"])),
                np.radians(double_pixels["dec_deg"]),
                c=double_pixels["prob"],
                vmin=0.0,
                vmax=max(self.data[self.key]),
                s=size,
            )

        red_patch = mpatches.Patch(color="red", label="Not observed")
        gray_patch = mpatches.Patch(color="gray", label="Observed once")
        violet_patch = mpatches.Patch(
            color="green", label="Observed Galactic Plane (|b|<10)"
        )
        plt.legend(handles=[red_patch, gray_patch, violet_patch])

        message = (
            "In total, {0:.1f} % of the contour was observed at least once.\n"
            "This estimate includes {1:.1f} % of the contour "
            "at a galactic latitude <10 deg.\n"
            "In total, {2:.1f} % of the contour was observed at least twice. \n"
            "In total, {3:.1f} % of the contour was observed at least twice, "
            "and excluding low galactic latitudes.\n"
            "These estimates account for chip gaps.".format(
                100.0 * coverage_df.query("n_det_class > 0")["prob"].sum(),
                100.0 * coverage_df.query("in_plane & n_det_class > 0")["prob"].sum(),
                100.0 * coverage_df.query("n_det_class == 2")["prob"].sum(),
                100.0 * coverage_df.query("n_det_class == 2 & ~in_plane")["prob"].sum(),
            )
        )

        n_pixels = len(coverage_df.query("n_det_class > 0 & prob > 0.0"))
        n_double = len(coverage_df.query("n_det_class == 2 & prob > 0.0"))
        n_plane = len(coverage_df.query("in_plane & n_det_class > 0 & prob > 0.0"))

        self.healpix_area = self.pixel_area * n_pixels
        self.double_extragalactic_area = self.pixel_area * n_double
        plane_area = self.pixel_area * n_plane

        self.overlap_fields = overlapping_fields

        self.logger.info(
            f"{n_pixels} pixels were covered, covering approximately "
            f"{self.healpix_area:.2g} sq deg."
        )
        self.logger.info(
            f"{n_double} pixels were covered at least twice (b>10), "
            f"covering approximately {self.double_extragalactic_area:.2g} sq deg."
        )
        self.logger.info(
            f"{n_plane} pixels were covered at low galactic latitude, "
            f"covering approximately {plane_area:.2g} sq deg."
        )
        return fig, message